In [1]:
% cd /home/mayu-ot/durga/Experiments/loc_iparaphrasing/

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [2]:
import sys
sys.path.append('script/data/')
from converter import *

In [3]:
import pandas as pd
import tables
import numpy as np

In [4]:
from ipywidgets import IntProgress
from IPython.display import display

In [5]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [6]:
split = 'test'

def save_plclc_data(split):
    phr_file = 'data/pl-clc/phraseWords_%s' % split
    sentence_dir = '../../Dataset/Flickr30kEntities/Sentences/'

    fp = IntProgress(min=0, max=file_len(phr_file))
    display(fp)


    with open(phr_file, 'r') as f:
        items = []
        for line_i, line in enumerate(f):
            if line[:5] == 'image':
                im_id = line.split()[-1]
                prepro_sentences = []
            elif line[:3] == 'sen':
                entities = load_preprocessed(line)
                prepro_sentences.append(entities)
            elif line == '\n':
                # load original entities
                sentences = load_sentence(os.path.join(sentence_dir, im_id+'.txt'))

                for org_s, prepro_s in zip(sentences, prepro_sentences):
                    for entity in org_s:
                        id_category_key = '%i/%s'%(entity['id'], entity['category'])
                        if prepro_s is None:
                            entity['prepro_phrase'] = '<discarded-phrase>'  
                        elif id_category_key in prepro_s.keys():
                            entity['prepro_phrase'] = prepro_s[id_category_key].pop(0)
                        else:
                            entity['prepro_phrase'] = '<discarded-phrase>'

                for s1, s2 in combinations(sentences, 2):
                    for ent1, ent2 in product(s1, s2):
                        if (ent1['prepro_phrase'] == '<discarded-phrase>') or (ent2['prepro_phrase'] == '<discarded-phrase>'):
                            continue

                        if (ent1['prepro_phrase'] == ent2['prepro_phrase']):
                            continue

                        y_true = (ent1['id'] == ent2['id']) and (ent1['category'] == ent2['category'])
                        items.append([im_id,
                                     ent1['cap_i'],
                                     ent2['cap_i'],
                                     ent1['id'],
                                     ent2['id'],
                                     ent1['category'],
                                     ent2['category'],
                                     ent1['prepro_phrase'],
                                     ent2['prepro_phrase'],
                                     ent1['org_phrase'],
                                     ent2['org_phrase'],
                                     y_true])
                fp.value = line_i

    df = pd.DataFrame(items, columns=['image', 'cap_id1', 'cap_id2', 'phr_id1', 'phr_id2', 'category1', 'category2', 'phrase1', 'phrase2', 'org_phrase1', 'org_phrase2', 'ytrue'])
    df.to_csv('data/pl-clc/phrase_pair_%s.csv'%split)

In [9]:
save_plclc_data('train')

IntProgress(value=0, max=208467)

In [10]:
def save_localization_data(split):
    in_file = 'data/pl-clc/entity_region_scores.%sSplit.txt' % split

    fp = IntProgress(min=0, max=file_len(in_file))
    display(fp)

    with open(in_file, 'r') as f:
        items = []
        for line_i, line in enumerate(f):

            if line[:3] == 'img':
                im_id, s_id, ent_id, group, phrase = parse.parse('img_{}:sen_{:d}:entity_{:d}/{}:{} ', line)

            elif line[:3] == 'box':
                box_id, score, xmin, ymin, xmax, ymax = parse.parse('box: {:d}, score: {:f}, coordinate: {:f}, {:f}, {:f}, {:f}, ',line)
                items.append([im_id, s_id, ent_id, group, phrase, box_id, xmin, ymin, xmax, ymax, score])
            else:
                fp.value = line_i
                pass

    df = pd.DataFrame(items, columns=['image', 'cap_id', 'phr_id', 'category', 'org_phrase', 'bbox', 'xmin', 'ymin', 'xmax', 'ymax', 'score'])
    df.to_csv('data/pl-clc/localization_%s.csv' % split)

In [17]:
save_localization_data('train')

IntProgress(value=0, max=13671232)

In [14]:
def save_plclc_data_wt_bbox(split):
    df = pd.read_csv('data/pl-clc/phrase_pair_%s.csv' % split)
    loc_df = pd.read_csv('data/pl-clc/localization_%s.csv' % split)

    bbox_list = []

    fp = IntProgress(min=0, max=len(df))
    display(fp)

    cur_img = 0

    for i, row in df.iterrows():

        if row['image'] != cur_img:
            sub_df = loc_df[loc_df.image == row['image']]
            cur_img = row['image']

        cap_id1 = row['cap_id1']
        phr_id1 = row['phr_id1']
        phr1 = row['org_phrase1']
        cat1 = row['category1']

        cap_id2 = row['cap_id2']
        phr_id2 = row['phr_id2']
        phr2 = row['org_phrase2']
        cat2 = row['category2']

        res1 = sub_df[(sub_df.cap_id == cap_id1 + 1) & (sub_df.phr_id == phr_id1) & (sub_df.category == cat1)]

        if pd.unique(res1.org_phrase).size > 1:
            res1 = res1[res1.org_phrase == phr1]

        res2 = sub_df[(sub_df.cap_id == cap_id2 + 1) & (sub_df.phr_id == phr_id2) & (sub_df.category == cat2)]

        if pd.unique(res2.org_phrase).size > 1:
            res2 = res2[res2.org_phrase == phr2]


        bbox1 = res1.bbox.iloc[0] if len(res1) else 0
        bbox2 = res2.bbox.iloc[0] if len(res2) else 0
        bbox_list.append((bbox1 - 1, bbox2 - 1))

        fp.value = i

    bbox_arr = np.array(bbox_list)
    bbox_df = pd.DataFrame(bbox_arr, columns=['roi1', 'roi2'])
    pd.concat([df, bbox_df], axis=1).to_csv('data/pl-clc/phrase_pair_wt_plclcbbox_%s.csv'%split)

In [18]:
save_plclc_data_wt_bbox('train')

IntProgress(value=0, max=2383741)

KeyboardInterrupt: 